In [3]:
from ultralytics import YOLO
import mediapipe as mp
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader, random_split
import cv2
import numpy as np
import os
import random
import platform
import yaml
from tqdm import tqdm
import gc
from torch.cuda import empty_cache

if platform.system() == "Darwin":
    print("your system is mac os")
    device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
else:
    print("your system is cuda")
    device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

your system is cuda


In [14]:
training_epochs = 100
patience = 30
batch = 4
imgsz = 640
name = "yolov9_epoch100"

In [15]:
yolo_model = YOLO("yolov9c-seg.pt").to(device)

yolo_model.train(data="/home/pepsi/Downloads/road_dataset_yolo/data.yaml", epochs=training_epochs, patience=patience, batch=batch, imgsz=imgsz, name=name)

New https://pypi.org/project/ultralytics/8.3.130 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=segment, mode=train, model=yolov9c-seg.pt, data=/home/pepsi/Downloads/road_dataset_yolo/data.yaml, epochs=100, time=None, patience=30, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=yolov9_epoch100, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show

train: Scanning /home/pepsi/Downloads/road_dataset_yolo/labels/train.cache... 1252 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1252/1252 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 324.8±102.9 MB/s, size: 114.7 KB)


val: Scanning /home/pepsi/Downloads/road_dataset_yolo/labels/val.cache... 314 images, 0 backgrounds, 0 corrupt: 100%|██████████| 314/314 [00:00<?, ?it/s]


Plotting labels to runs/segment/yolov9_epoch100/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000625, momentum=0.9) with parameter groups 163 weight(decay=0.0), 174 weight(decay=0.0005), 173 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/segment/yolov9_epoch100
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        314        836      0.716      0.462      0.542      0.352      0.732      0.486      0.564      0.334

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      3/100      4.32G      1.001      1.124      1.128      1.221         19        640: 100%|██████████| 313/313 [01:46<00:00,  2.94it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 40/40 [00:08<00:00,  4.77it/s]


                   all        314        836      0.368      0.651      0.535      0.358      0.393      0.607      0.523      0.336

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      4/100      4.37G      1.019       1.02      1.036      1.213         19        640:  31%|███▏      | 98/313 [00:35<01:17,  2.78it/s]


KeyboardInterrupt: 

In [13]:
classes = ['solid','center', 'dotted', 'stopline', 'crosswlaker', 'pinky', 'speedlimit_30', 'speedlimit_60', 'redlight', 'yellowlight', 'greenlight', 'pedesrian']

yolo_best_model_path = "/home/pepsi/dev_ws/ros-repo-5/admin/src/ai_server/ai_train/runs/segment/yolov9_epoch150/weights/best.pt"
yolo_best_model = YOLO(yolo_best_model_path).to(device)

metrics = yolo_best_model.val()

YOLOv9c-seg summary (fused): 169 layers, 27,633,780 parameters, 0 gradients, 157.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1257.9±405.3 MB/s, size: 37.4 KB)


val: Scanning /home/pepsi/Downloads/road_dataset_yolo/labels/val.cache... 314 images, 0 backgrounds, 0 corrupt: 100%|██████████| 314/314 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:15<00:00,  1.28it/s]


                   all        314        836      0.892      0.933      0.939      0.797      0.898      0.937      0.943      0.731
                 solid        240        240      0.998          1      0.995      0.966      0.998          1      0.995      0.915
                center        184        184      0.988          1      0.995       0.98      0.988          1      0.995      0.883
                dotted        122        219      0.812      0.808      0.882      0.657      0.813      0.808      0.876      0.548
              stopline         47         60      0.801       0.75      0.764      0.603      0.819      0.767      0.781      0.613
             crosswalk         22         25       0.86        0.8      0.875      0.478      0.904       0.84      0.916      0.492
                 pinky         20         20      0.864       0.95      0.947      0.849      0.869       0.95      0.947      0.877
         speedlimit_30         19         19      0.863      0.993   